# Ensemble dataset

In [6]:
import pandas as pd

# Load the provided dataset
file_path = r'D:\Prab\IPS-Project\synthetic_data_point\Noon-RSSI\Ensemble_Noon_synthetic_data.csv'
data = pd.read_csv(file_path)
data.head()

,Latitude,Longitude,CMKL-Guest_2a:3f:0b:56:e9:15,CMKL_2e:3f:1b:56:e6:18,CMKL-TV_26:3f:0b:56:e6:18,CMKL-TV_26:3f:0b:56:e9:00,CMKL-TV_26:3f:0b:56:e9:03,CMKL-Guest_2a:3f:0b:56:e6:18,CMKL-TV_26:3f:0b:57:fa:37,KMITL-IoT_9c:50:ee:83:b2:84,...,CMKL_2e:3f:1b:56:e8:fb,CMKL-Guest_2a:3f:1b:56:e8:f1,CMKL-Guest_2a:3f:1b:56:e8:f7,CMKL-TV_26:3f:1b:56:e8:f1,CMKL-TV_26:3f:1b:56:e8:f7,_12:3f:1b:56:e6:18,CMKL-Guest_e2:55:a8:26:73:ef,CMKL_2c:3f:0b:56:e9:15,CMKL-Guest_2a:3f:0b:57:fa:37,KMITL-IoT_9c:50:ee:83:b4:44
0,13.7279,100.778282,-93.353546,-62.299463,-74.907603,-94.519601,-80.109546,-74.008237,-91.730278,-81.676786,...,-90.752058,-87.814967,-71.012308,-87.050971,-70.951503,-94.395917,-96.493747,-95.113937,-88.214555,-88.211468
1,13.7279,100.778282,-93.353211,-62.301404,-74.909968,-94.516645,-80.110509,-74.010520,-91.728249,-81.673683,...,-90.748998,-87.811625,-71.010893,-87.047627,-70.950096,-94.396971,-96.492695,-95.113700,-88.212859,-88.210710
2,13.7279,100.778282,-93.352877,-62.303345,-74.912332,-94.513689,-80.111471,-74.012803,-91.726221,-81.670580,...,-90.745937,-87.808282,-71.009479,-87.044282,-70.948689,-94.398025,-96.491643,-95.113463,-88.211162,-88.209953
3,13.7279,100.778282,-93.352542,-62.305286,-74.914697,-94.510734,-80.112433,-74.015087,-91.724192,-81.667477,...,-90.742877,-87.804939,-71.008064,-87.040937,-70.947281,-94.399080,-96.490591,-95.113227,-88.209465,-88.209196
4,13.7279,100.778282,-93.352208,-62.307227,-74.917062,-94.507778,-80.113396,-74.017370,-91.722164,-81.664374,...,-90.739816,-87.801596,-71.006650,-87.037592,-70.945874,-94.400134,-96.489540,-95.112990,-88.207769,-88.208438


In [1]:
import os
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

model_dir = r'D:\Prab\IPS-Project\synthetic_data_point\Noon-RSSI\Noon_saved_models'

# Assume 'data' is previously loaded as a DataFrame
X_new = data.drop(['Latitude', 'Longitude'], axis=1)
y_new_multi = data[['Latitude', 'Longitude']]
y_new_sep = {
    'Latitude': data['Latitude'],
    'Longitude': data['Longitude']
}

results = {
    'MultiOutput Approach': {},
    'Separate Approach': {}
}

# Simplify model names and structure data by model first
def simplify_name(filename):
    parts = filename.replace('.joblib', '').split('_')
    return parts[0]

# Load and predict with each model
for filename in os.listdir(model_dir):
    if filename.endswith('.joblib'):
        model_path = os.path.join(model_dir, filename)
        model = joblib.load(model_path)
        model_name = simplify_name(filename)
        search_type = "Grid Search" if "grid" in filename.lower() else "Randomized Search"

        if 'multioutput' in filename.lower():
            predictions = model.predict(X_new)
            rmse = np.sqrt(mean_squared_error(y_new_multi, predictions))
            best_params = model.estimator.get_params() if hasattr(model, 'estimator') else model.get_params()
            if model_name not in results['MultiOutput Approach']:
                results['MultiOutput Approach'][model_name] = {}
            results['MultiOutput Approach'][model_name][search_type] = {'RMSE': rmse, 'Best Params': best_params}
        else:
            for target in ['Latitude', 'Longitude']:
                if target.lower() in filename.lower():
                    predictions = model.predict(X_new)
                    rmse = np.sqrt(mean_squared_error(y_new_sep[target], predictions))
                    if model_name not in results['Separate Approach']:
                        results['Separate Approach'][model_name] = {}
                    if target not in results['Separate Approach'][model_name]:
                        results['Separate Approach'][model_name][target] = {}
                    results['Separate Approach'][model_name][target][search_type] = {'RMSE': rmse, 'Best Params': model.get_params()}

# Print formatted results
print("Results for MultiOutput Approach:")
for model, searches in results['MultiOutput Approach'].items():
    print(f"\n  Model: {model}")
    for search_type, details in searches.items():
        print(f"    {search_type}: RMSE: {details['RMSE']:.16f}, Best Params: {details['Best Params']}")

print("\nResults for Separate Approach:")
for model, targets in results['Separate Approach'].items():
    print(f"\n  Model: {model}")
    for target, searches in targets.items():
        print(f"    Target: {target}")
        for search_type, details in searches.items():
            print(f"      {search_type}: RMSE: {details['RMSE']:.16f}, Best Params: {details['Best Params']}")


NameError: name 'data' is not defined

# GANs dataset

In [1]:
import pandas as pd

# Load the provided dataset
file_path = r'D:\Prab\IPS-Project\synthetic_data_point\Noon-RSSI\GANs_synthetic_samples.csv'
data = pd.read_csv(file_path)
data.head()

,CMKL-Guest_2a:3f:0b:56:e9:15,CMKL_2e:3f:1b:56:e6:18,CMKL-TV_26:3f:0b:56:e6:18,CMKL-TV_26:3f:0b:56:e9:00,CMKL-TV_26:3f:0b:56:e9:03,CMKL-Guest_2a:3f:0b:56:e6:18,CMKL-TV_26:3f:0b:57:fa:37,KMITL-IoT_9c:50:ee:83:b2:84,CMKL-TV_26:3f:0b:56:e8:f7,CMKL-Guest_2a:3f:1b:56:e9:2a,...,CMKL-Guest_2a:3f:1b:56:e8:f7,CMKL-TV_26:3f:1b:56:e8:f1,CMKL-TV_26:3f:1b:56:e8:f7,_12:3f:1b:56:e6:18,CMKL-Guest_e2:55:a8:26:73:ef,CMKL_2c:3f:0b:56:e9:15,CMKL-Guest_2a:3f:0b:57:fa:37,KMITL-IoT_9c:50:ee:83:b4:44,Latitude,Longitude
0,-96.227910,-57.061890,-73.389890,-71.554270,-91.636330,-79.684494,-83.527060,-74.249940,-72.045130,-67.639260,...,-54.661240,-67.272415,-66.740340,-86.35835,-94.834366,-93.996950,-82.169170,-95.630670,13.727912,100.778330
1,-97.416374,-50.763115,-78.357440,-56.724236,-112.326040,-83.460846,-69.996720,-53.847950,-62.076744,-59.674812,...,-51.438820,-53.606457,-56.028374,-91.55781,-88.009580,-84.753980,-68.481895,-92.908580,13.727929,100.778380
2,-92.974450,-63.538673,-76.313896,-95.551130,-68.762440,-71.938200,-92.518555,-100.115680,-80.119980,-65.265790,...,-63.029907,-93.456440,-72.261550,-89.71870,-95.781330,-96.453030,-90.416960,-93.087875,13.727912,100.778290
3,-96.534460,-62.744780,-86.133644,-69.749470,-95.317120,-90.288330,-77.241066,-62.288986,-71.037110,-68.673515,...,-57.085594,-59.604137,-66.180740,-96.52677,-98.905110,-90.456410,-75.483340,-96.758484,13.727867,100.778350
4,-85.896490,-64.456480,-82.025740,-99.709260,-60.442608,-76.242140,-88.451970,-95.773630,-69.629870,-56.856556,...,-61.951740,-89.526720,-73.117350,-96.95759,-93.749580,-90.803566,-88.783745,-75.708600,13.727902,100.778305


In [2]:
import os
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

model_dir = r'D:\Prab\IPS-Project\synthetic_data_point\Noon-RSSI\Noon_saved_models'

# Assume 'data' is previously loaded as a DataFrame
X_new = data.drop(['Latitude', 'Longitude'], axis=1)
y_new_multi = data[['Latitude', 'Longitude']]
y_new_sep = {
    'Latitude': data['Latitude'],
    'Longitude': data['Longitude']
}

results = {
    'MultiOutput Approach': {},
    'Separate Approach': {}
}

# Simplify model names and structure data by model first
def simplify_name(filename):
    parts = filename.replace('.joblib', '').split('_')
    return parts[0]

# Load and predict with each model
for filename in os.listdir(model_dir):
    if filename.endswith('.joblib'):
        model_path = os.path.join(model_dir, filename)
        model = joblib.load(model_path)
        model_name = simplify_name(filename)
        search_type = "Grid Search" if "grid" in filename.lower() else "Randomized Search"

        if 'multioutput' in filename.lower():
            predictions = model.predict(X_new)
            rmse = np.sqrt(mean_squared_error(y_new_multi, predictions))
            best_params = model.estimator.get_params() if hasattr(model, 'estimator') else model.get_params()
            if model_name not in results['MultiOutput Approach']:
                results['MultiOutput Approach'][model_name] = {}
            results['MultiOutput Approach'][model_name][search_type] = {'RMSE': rmse, 'Best Params': best_params}
        else:
            for target in ['Latitude', 'Longitude']:
                if target.lower() in filename.lower():
                    predictions = model.predict(X_new)
                    rmse = np.sqrt(mean_squared_error(y_new_sep[target], predictions))
                    if model_name not in results['Separate Approach']:
                        results['Separate Approach'][model_name] = {}
                    if target not in results['Separate Approach'][model_name]:
                        results['Separate Approach'][model_name][target] = {}
                    results['Separate Approach'][model_name][target][search_type] = {'RMSE': rmse, 'Best Params': model.get_params()}

# Print formatted results
print("Results for MultiOutput Approach:")
for model, searches in results['MultiOutput Approach'].items():
    print(f"\n  Model: {model}")
    for search_type, details in searches.items():
        print(f"    {search_type}: RMSE: {details['RMSE']:.16f}, Best Params: {details['Best Params']}")

print("\nResults for Separate Approach:")
for model, targets in results['Separate Approach'].items():
    print(f"\n  Model: {model}")
    for target, searches in targets.items():
        print(f"    Target: {target}")
        for search_type, details in searches.items():
            print(f"      {search_type}: RMSE: {details['RMSE']:.16f}, Best Params: {details['Best Params']}")


Results for MultiOutput Approach:

  Model: DecisionTree
    Grid Search: RMSE: 0.0000383775905865, Best Params: {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 42, 'splitter': 'best'}
    Randomized Search: RMSE: 0.0000360844710990, Best Params: {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 25, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 11, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 42, 'splitter': 'best'}

  Model: GradientBoosting
    Grid Search: RMSE: 0.0000356989536346, Best Params: {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 5, 'max_features': None, 'max_le